## Install Packages

In [1]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")
		Just
With SwiftPM flags: []
Working in: /tmp/tmpgkoma8im
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 2.41s
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Loading library...
Installation complete!


## Use Packages & Flag Exportable Cells
Taken from fastai's `00_load_data` notebook.

In [2]:
// export
import Foundation
import Just
import Path

In [3]:
//export
public func downloadFile(_ url: String, dest: String?=nil, force: Bool=false){
    let dest_name = (dest ?? (Path.cwd/url.split(separator: "/").last!).string)
    let url_dest = URL.init(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if (force || !Path(dest_name)!.exists){
        print("Downloading \(url)...")
        if let cts = Just.get(url).content{
            do    {try cts.write(to: URL.init(fileURLWithPath:dest_name))}
            catch {print("Can't write to \(url_dest).\n\(error)")}
        } else {print("Can't reach \(url)")}
    }
}

In [4]:
downloadFile("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")

## Export

List dependencies

In [5]:
%listInstalledPackages

Package Just @ 0.7.1 from https://github.com/JustHTTP/Just
Package Path @ 0.16.2 from https://github.com/mxcl/Path.swift


Prepare export directory

In [6]:
%createPackageSpec ExportedNotebooks

Prepared Swift Package skeleton at ExportedNotebooks

Use fastai's cell export code

In [7]:
// export
let target = "ExportedNotebooks"
public func notebookToScript(fname: String){
    let url_fname = URL.init(fileURLWithPath: fname)
    let last = fname.lastPathComponent
    let out_fname = (url_fname.deletingLastPathComponent().appendingPathComponent(target, isDirectory: true)
                     .appendingPathComponent("Sources", isDirectory: true)
                     .appendingPathComponent(target, isDirectory: true).appendingPathComponent(last)
                     .deletingPathExtension().appendingPathExtension("swift"))
    do{
        let data = try Data.init(contentsOf: url_fname)
        let jsonData = try! JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells{
            if let source = cell["source"] as? [String]{
                if source.isEmpty {continue}
                if source[0].range(of: #"^\s*//\s*export\s*$"#, options: .regularExpression) != nil{
                    module.append("\n" + source[1...].joined() + "\n")
                }
            }
        }
        try? module.write(to: out_fname, atomically: false, encoding: .utf8)
    } catch {print("Can't read the content of \(fname)")}
}

In [8]:
notebookToScript(fname: (Path.cwd / "install_and_export.ipynb").string)